# Script du modèle CNN-Transformer

In [ ]:
import torch
import torch.nn as nn

In [ ]:
# -----------------------------
# Dataset
# -----------------------------
X_tensor = torch.tensor(X_seq, dtype=torch.float32)
y_tensor = torch.tensor(y_seq, dtype=torch.long)

dataset = TensorDataset(X_tensor, y_tensor)
loader  = DataLoader(dataset, batch_size=64, shuffle=True)



class CNNTransformer(nn.Module):
    """
    x shape attendu : (batch, seq_len, feature_dim)
    """
    def __init__(self, feature_dim, num_classes, d_model=128, nhead=4, num_layers=4):
        super().__init__()

        # ----- CNN -----
        # On applique un CNN 1D sur la dimension temporelle :
        # (batch, seq_len, feature_dim) -> (batch, feature_dim, seq_len)
        self.conv = nn.Sequential(
            nn.Conv1d(feature_dim, d_model, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(d_model, d_model, kernel_size=3, padding=1),
            nn.ReLU()
        )

        # ----- Transformer -----
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # ----- Classification -----
        self.cls_head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, num_classes)
        )

    def forward(self, x):
        # x : (B, L, F) -> conv attend (B, F, L)
        x = x.transpose(1, 2)

        x = self.conv(x)          # (B, d_model, L)
        x = x.transpose(1, 2)     # retour (B, L, d_model)

        x = self.transformer(x)   # encoder
        x = x[:, -1, :]           # dernier token

        return self.cls_head(x)

# -----------------------------
# Instanciation
# -----------------------------
model = CNNTransformer(
    feature_dim=X_seq.shape[2],
    num_classes=5,
    d_model=128,
    nhead=4,
    num_layers=4
).to(device)

# -----------------------------
# Entraînement (identique)
# -----------------------------
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)

        pred = model(xb)
        loss = criterion(pred, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("epoch", epoch, "loss", loss.item())